# Voluntary Problem Set

This problem set allows you to play around with concepts from class and to solve some smaller subproblems on your own.

Additional guidance is provided for concepts that did not make into our class discussion. 

# Topic: Style Investing and Unconditional (backward-looking) Market Prices of Factor Premiums in US Equity

$$
\\
$$

Long time ago, Fama and MacBeth proposed a two-step Least Squares approach to estimate the amount of **systematic risk (beta)** and its **market price of risk (lambda)** in two consecutive steps.

$$
\quad E[ r_{i,t}] - r_t = \alpha + \lambda^{f} \times \beta_i^{f}   
$$

where $\lambda^{f}$ is the unconditional market price of $f$-risk.

To estimate that **unconditional market price of risk**, Fama and MacBeth propose a **two step estimation procedure:**

- First, regress time-series of individual exess returns onto the risk factors to get betas of assets (i.e. time-series regressions). 

- Second, for each time point, regress the cross-section of asset's excess returns onto their beta estimate from step 1. (i.e. cross-sectional regression)  

$$
\\
$$

**We let data speak whether there is evidence for FF7 risk premia in daily US stock returns.**

## Fama-MacBeth Procedure Explained. Application: CAPM, 377 Stocks and T time points
$$
\\
$$

We use Fama-MacBeth regressions to identify $\lambda$ and $\beta$ using a two step approach, here applied to the CAPM
$$
\\
$$


**Step 1:** time-series regressions for each individual stock returns r to recover the stock's  $\beta$
$$
r_{t=1,...,T} - r_{f} = \alpha + \beta (r_{MKT,t}-r_{f}) + \epsilon_{t=1,...,T}
$$
$$
\\
$$

**Step 2:** cross-sectional regressions for each time point t to recover $\lambda_f$
$$
r_{i=1,...,377} - r_f = \alpha + \lambda \times \beta_{i=1,...,377} + \epsilon_{i=1,...,377}
$$
$$
\\
$$

**Step 3:**
$$
\lambda_{FMacB} = \frac{1}{T} \sum_t \lambda_t, \qquad \text{and} \qquad t(\lambda_{FMacB}) = \frac{\lambda_{FMacB}}{std(\lambda_t)/ \sqrt{T}}
$$

In [3]:
# R_T: The return panel. T x I for I assets and T periods
# R_f: Risk free rate. T x 1

# Step 1: Regress for each i y = R_T_i - r_f and X = FF7 -> B_i_T

# Step 2: Regress for each y = R_i_T; X = B_i_T -> lambda


# Your Learning Points

you will learn  

- about the industry standard to estimate factor premiums, i.e. Fama-MacBeth procedure
$$
\\
$$

- that this standard approach has its problems when applying it to daily return data. 
$$
\\
$$

- that there is weak evidence for factor premiums in daily returns. This motivates us to go beyond the traditional factor investment approach
$$
\\
$$

- Chances are high you will meet such an exercise during an interview with the financial industry

# Background Knowledge (not necessary, yet helpful to grasp the context, for solving the Problem Set)
$$
\\
$$

**Students asking about FamaMacBeth**

https://quant.stackexchange.com/questions/46746/rationale-of-fama-macbeth-procedure/46764


# Background Knowledge (not necessary to solve Problem Set):
$$
\\
$$

**A worked-out example for Fama-MacBeth is here** unfortunately with a copy / paste error in the visuals, yet the audio is correct

https://www.youtube.com/watch?v=9sIFMRDpxyI
 

# Task A.0 [Degree of Difficulty: Beginner]: 
$$
\\
$$

**Notice:**  **R_d.csv** contains merged data for FF7, Rf and SP500 constituents 

$$
\\
$$

### A.0: Fama-MacBeth

Program a Class FMacBeth() that implements the Fama-MacBeth procedure for the FF7 factor model (i.e. MKT, SMB, HML, RMW, CMA, Reversal, Momentum).  The output of the class shall be the estimates for the factor exposure (beta), unconditional market price of factor risk (lambda) and the respective adj-R-squares and t-stats. 


$$
\\
$$
**Hint:**
H_A_1: The attached file "Helper_FamaMacBeth.ipynb" contains a solution to the task
$$
\\
$$








# Task A.0 [Degree of Difficulty: Advanced]:

$$
\\
$$

**Notice:**  **R_d.csv** contains merged data for FF7, Rf and SP500 constituents 

$$
\\
$$

### A.0: Fama-MacBeth

Program a Class FMacBeth() that implements the Fama-MacBeth procedure for the FF7 factor model (i.e. MKT, SMB, HML, RMW, CMA, Reversal, Momentum).  The output of the class shall be the estimates for the factor exposure (beta), unconditional market price of factor risk (lambda) and the respective adj-R-squares and t-stats.  

$$
\\
$$

**Hint:**
H_A_0:  (i) initialize the class with the panel of returns, panel of realized factor returns and the risk-free rate, (ii) one function of the class takes care of the time-series regression to compute beta, another function takes these betas as known features for the cross-sectional regressions, (iii) store adj-R2 and t-stats for each regression and class members.
$$
\\
$$

 


 

# Task A.0 [Degree of Difficulty: Expert]:


$$
\\
$$

**Notice:**  **R_d.csv** contains merged data for FF7, Rf and SP500 constituents 

$$
\\
$$

### A.0: Fama-MacBeth

Program a Class FMacBeth() that implements the Fama-MacBeth procedure for the FF7 factor model (i.e. MKT, SMB, HML, RMW, CMA, Reversal, Momentum).  The output of the class shall be the estimates for the factor exposure (beta), unconditional market price of factor risk (lambda) and the respective adj-R-squares and t-stats.
 
 
 

In [335]:
import pandas as pd
import statsmodels.api as sm
import numpy as np


class FMacBeth:

    def __init__(self, r: pd.DataFrame, x: pd.DataFrame, rf: pd.DataFrame):

        self.r = r  # R_T: The return panel. T x I for I assets and T periods
        self.rf = rf                     # R_f: Risk free rate. T x 1
        self.x = x                       # E.g. FF7 Matrix

        self.summary_table = None
        self.B = None
        self.lambda_T = None
        self.lambda_MB = None
        self.R_squared = None
        self.t_stat = None
        self.lambda_r2 = None
        self.beta_r2 = None

    def fit(self):
        # First pass
        self.B, self.beta_r2 = self._get_betas_for_assets()
        # Second pass
        self.lambda_T, self.lambda_r2 = self._get_lambda_per_t()

        T = len(self.lambda_T)
        self.lambda_MB = np.mean(self.lambda_T)

        self.t_stat = self.lambda_MB / (np.std(self.lambda_T) / np.sqrt(T))

    def _get_betas_for_assets(self):
        B_I_T = []
        r_sq = []
        for i in self.r:
            excess_return = self.r[i] - self.rf
            
            X_ = sm.add_constant(self.x)  # Add alpha as alpha = (1 1 1 ... 1 1 1)
            
            res = sm.GLS(excess_return, X_).fit()
            B_i_T = res.params.values[1:]
            B_I_T.append(B_i_T)
            r_sq.append(res.rsquared_adj)
        return pd.DataFrame(B_I_T, index=self.r.columns), pd.DataFrame(r_sq, index=self.r.columns)

    def _get_lambda_per_t(self):
        lambda_T_f = []
        r_sq = []
        for t in range(self.r.shape[0]):
            excess_return = self.r.iloc[t,:] - self.rf.iloc[t]
            
            X_ = sm.add_constant(self.B)         # Add alpha as alpha = (1 1 1 ... 1 1 1)
            res = sm.GLS(excess_return, X_).fit()
            lambda_t_f = res.params.values[1:]
            lambda_T_f.append(lambda_t_f)
            r_sq.append(res.rsquared_adj)

        return pd.DataFrame(lambda_T_f, index=self.r.index), pd.DataFrame(r_sq, index=self.r.index)


In [336]:
df = pd.read_csv("R_d.csv", parse_dates=True, index_col=0)

ff7 = df.iloc[:,0:8].drop("RF_x", axis=1)
rf = df.iloc[:,5]
r = df.iloc[:,8:]


fmb = FMacBeth(r, ff7, rf)

fmb.fit()


In [337]:
# A_1: Look at the betas using the print() command. Write down the betas for company XLNX
np.round(fmb.B.loc["XLNX",:], 2)

0    1.17
1    0.06
2   -0.80
3   -1.10
4   -0.18
5   -0.08
6   -0.28
Name: XLNX, dtype: float64

In [338]:
# A_2: For which 'investment style' is XLNX well suited?

# strong business cycle sensitivity: ['yes'/'no']
a2_q1 = 'yes'

# growth firm: ['yes'/'no']
a2_q2 = 'yes'

# lots of operating profit: ['yes'/'no']
a2_q3 = 'no'

# agressive investments: ['yes'/'no']
a2_q4 = 'no'

# short-term reversal return strategy: ['yes'/'no']
a2_q5 = 'no'

# positive momentum: ['yes'/'no']
a2_q6 = 'no'

In [339]:
# Var explained
print("Variance explained by ff7", str(round(fmb.beta_r2.T["XLNX"][0], 2)))
      
# Diversifiable
print("Diversifiable", str(round(1 - fmb.beta_r2.T["XLNX"][0], 2)))

Variance explained by ff7 0.45
Diversifiable 0.55


In [340]:
########### Task A.3 ###########

# How much of the variance in the XLNX's stock return is explained by the 7 factors?


variance_explained_FF7_XLNX = 0.45


########### Task A.4 ###########

# What fraction of risk in XLNX is diversifiable, assuming the FF7 model holds?
variance_diversifiable_FF7_XLNX = 0.55

In [341]:
########### Task A.5 ###########

# Which firm is best explained by the FF7 factors (give ticker as answer)? 
# How large is its respective adj-R2? 

In [342]:
idx = np.argmax(fmb.beta_r2)
fmb.beta_r2.iloc[idx]

0    0.706979
Name: JPM, dtype: float64

In [343]:
round(fmb.B.loc["JPM"], 2)

0    1.23
1   -0.35
2    1.30
3   -0.55
4   -0.55
5   -0.02
6   -0.20
Name: JPM, dtype: float64

In [344]:
best_fit_TSreg = 'JPM'
best_R2_TSreg = 0.71
beta_A5_MKT = 1.23
beta_A5_SMB = -0.35
beta_A5_HML = 1.30
beta_A5_RMW = -0.55
beta_A5_CMA = -0.55
beta_A5_Rev = -0.02
beta_A5_Mom = -0.20

In [345]:
#### Characterize its 'investment style' in terms of its beta coefficients and in terms of a qualitative assessment:
#'The firm whose return is best explained by FF is characterized as follows: a ['large'/'small']
a5_q1 = 'large'

#...firm with a ['cyclical'/'countercyclical']
a5_q2 = 'cyclical'

#...business model that based on balance sheet fundamentals can be classified as rather ['expensive'/'cheap']
a5_q3 = 'cheap'

#...with ['lots of'/'low']
a5_q4 = 'low'

#...operating profits and ['aggressive'/'passive']
a5_q5 = 'aggressive' # **solution says passive, but why? CMA is negative**

#...investment strategies and ['positive'/'negative']
a5_q6 = 'negative'

#...return momentum'


In [346]:
np.mean(fmb.lambda_T) * 252

0    0.032812
1    0.058918
2   -0.037614
3    0.024524
4   -0.032094
5   -0.105923
6    0.305056
dtype: float64

In [347]:
# A_6. State the Fama-MacBeth market price of risks (annualized in percent) and respective t-stats

fmb_ann = fmb.lambda_MB.apply(lambda x: round(x * 252, 2))
fmb_ann

0    0.03
1    0.06
2   -0.04
3    0.02
4   -0.03
5   -0.11
6    0.31
dtype: float64

In [348]:
round(fmb.t_stat, 2)

0    0.50
1    1.93
2   -1.14
3    0.92
4   -1.09
5   -1.19
6    3.84
dtype: float64

# Additional Tasks


### A_1: Look at the betas using the print() command. Write down the betas for company XLNX

 


### A_2: For which 'investment style' is XLNX well suited?

 


### A_3: How much of the variance in the XLNX's stock return is explained by the 7 factors? 

 
### A_4: What fraction of risk in XLNX is diversifiable, assuming the FF7 model holds?


### A_5: Which firm is best explained by the FF7 factors? How large is its respective adj-R2? Characterize its 'investment style' in terms of its beta coefficients and in terms of a qualitative assessment
 

### A_6. State the Fama-MacBeth market price of risks (annualized in percent) and respective t-stats 
 


### A_7. What is the main reason for these disappointing results?

 